# Introducción a Word Vectors

En muchas de las tareas de procesamiento del lenguaje, el objetivo es realizar alguna decisión sobre un texto. Por ejemplo, 

* Clasificación de emails en SPAM / NO SPAM: A partir de un email escrito, se desea saber si es un SPAM o no. 
* Análisis de sentimientos: Tarea en que se define si un texto escrito tiene alguna connotación negativa, positiva o neutra.
* Predicción de texto: A partir de una oración o texto, quiero predecir nuevas palabras y así generar nuevo texto.
* Correspondencia de información: Quiero buscar en un conjunto de palabras, la que esté más relacionada con el texto de entrada.
* Traducción automática: Quiero generar una serie de palabras que significan lo mismo que el texto de entrada, pero en otro idioma.

Para realizar todas estas tareas, muchas veces funciona bien la estrategia de que el algoritmo realice una extracción de características del texto y luego tome la decisión de clasificar (o la decisión que sea). Un método para extraer automáticamente las características del texto que demostró funcionar muy bien en este último tiempo es el de word vectors. Este método asigna un vector de $\mathbb{R}^n$ a cada palabra del vocabulario con el que se está trabajando, de manera que los vectores de las palabras con significados similares (desde el punto de vista de la lingüística) tengan características cuantificables similares. Los algoritmos que se van a ver a continuación sirven para obtener estas representaciones automáticamente, de manera que sea más fácil implementar el algoritmo completo.

**Objetivo**. El objetivo a continuación va a ser representar el significado de las palabras mediante vectores, es decir, elementos de un espacio vectorial $\mathbb{R}^n$ tradicional. 

**El Punto de vista de la lingüística**. El área de la lingüística que estudia el significado de las palabras se llama "semántica lingüística" (*lexical semantics*) y permite entender (e incluso cuantificar) algunas características del significado de las palabras. Por lo pronto, las más importantes son:

* El **sentido** de la palabra, que hace referencia a uno de los muchos significados que puede tener una palabra. Por ejemplo, el sentido de la palabra *mouse* cuando se refiere al roedor es distinto al que se refiere al controlador del cursor. 
* Los **sinónimos** de esa palabra, es decir las palabras que son distintas pero que tienen el mismo sentido. Debe notarse que para determinar si dos palabras tienen el mismo sentido, hay que mirar el contexto. 
* La **similitud** (*similarity*) y la **relación** (*relatedness*) con otras palabras, que son características que refieren al parecido entre sentidos. La primera, *similarity*, se refiere a la coincidencia de cualidades, mientras que  la segunda, *relatedness*, se refiere a si, por algún motivo cultural, las palabras suelen estar asociadas de alguna manera. Por ejemplo, las palabras "perro" y "gato" claramente no tienen el mismo sentido (probablemente para ninguno de sus contextos lo tenga), por lo que no califican como sinónimos. Sin embargo, tienen una similitud y una relación porque tienen características en común (ambos son animales domésticos). Por otro lado, las palabras "café" y "taza" (en su sentidos más tradicionales) no son similares, puesto que no comparten ninguna característica inmediata en común, pero están relacionadas porque se asocian inmediatamente entre sí por un motivo cultural. (¡Notemos cuán subjetivas son todas estas definiciones!) Probablemente, las palabras pertenecientes al mismo campo semántico tengan una relación, pero no todas ellas tengan una similitud. 
* **Cuadros semánticos** y **papeles semánticos**, que son características que permiten comprimir mucha información del significado de las oraciones en pocas palabras. Hay palabras que codifican un cuadro semántico por el hecho de que su sentido evoca a una situación particular. Por ejemplo, la acción de realizar una transacción comercial entre un vendedor y un comprador puede ser codificada en las palabras "vender", "comprar" o incluso "pagar". En este caso, el cuadro semántico de estas palabras sería el evento de la transacción comercial y podría obtenerse el papel semántico de esas palabras: "vendedor", "comprador", "pagador".
* La **connotación semántica** de las palabras, que son muy importantes en tareas de análisis de sentimientos. La connotación de una palabra se refiere a su significado afectivo que tiene esa palabra. 

Vamos a ver que características como el sentido de la palabra muchas veces quedan bien representadas por un vector de gran dimensión $(n = 50\!-\!\!1000)$ y que las herramientas algebráicas como el producto interno permiten cuantificar la similitud entre palabras.

# Representaciones del significado

La representación del significado de una palabra con un vector puede ser **denotacional** o **distribuida**. 

La primera, viene de una disciplina de ciencias de la computación que tiene por objetivo asignar una representación matemática a una expresión del lenguaje (en particular, un lenguaje de programación). Esta representación considera fundamentalmente la usabilidad (es decir que el vector que representa a la palabra sea útil, desambiguo y cómodo para usar en la práctica) y puede verse simplemente como una forma de codificación del lenguaje en un objeto matmático (en este caso, un vector). 

El segundo tipo de representación es la que hace uso de la hipótesis distribucional: "El significado de una palabra está determinado por su uso frente a un contexto". Es decir, se asume que si una palabra tiene un significado definido, eventualmente (o sea, con una gran cantidad de repeticiones) el significado de esa palabra va poder deducirse de su uso. 
Dentro de la representación distribucional, existen dos tipos de vectores: los dispersos (*sparse*), que se basan mayormente en métodos estadísticos tradicionales de conteo y estimación frecuentista, y los densos (*dense*), obtenidos generalmente con algoritmos de aprendizaje automático.

Todas las representaciones que se van a ver, salvo la que representa con vectores *one-hot*, son distribucionales, dado que las representacions denotacionales son simplemente una forma de codificación del texto. 

Los algoritmos de aprendizaje conocidos hasta ahora permiten aprender automáticamente representaciones distribucionales a partir de grandes cantidades de datos. Los primeros algoritmos que aparecieron se basaban en métodos de conteo de palabras, mientras que los que aparecieron con el surgimiento de la era *deep learning* suelen tener una base en estimar probabilidades con redes neuronales. A continuación se describirán un poco más detalladamente algunos de estos métodos. 


## Representación básica: codificación con vectores *one-hot*

La primera representación del significado de las palabras es a partir de vectores *one-hot*, la cual es más o menos intitiva. 

A partir de un texto, se define el vocabulario de palabras $V$, de tamaño $|V|$. Este incluye todas las palabras que aparecieron al menos una vez en el texto, ordenadas con algún criterio y sin repetirse. Con esto, es posible definir que

$$
h_j = 
\begin{bmatrix}
0 \\
\vdots \\
0 \\
1 \\
0 \\
\vdots \\
0
\end{bmatrix}
\in \mathbb{R}^{|V|}
$$

es un vector *one-hot* con su j-ésima coordenada igual a 1, y cero en otro lado. Por ejemplo, para un vocabulario $V=\{ casa, conjunto, el, diez, vacío, la, ellos, ellas, \ldots \}$, las palabras "conjunto" y "vacío" se representan por los vectores $h_2$ y $h_5$, respectivamente. También, podría definirse con este método el significado de una oración a partir de la suma (o el promedio) de los diferentes vectores. 

Si bien esta representación es muy directa de obtener y cómoda para trabajar con operaciones entre vectores, presenta muchas desventajas:

* No hay una representación del significado apreciable. Es decir, el vector $h_2$ representa a la palabra "conjunto", pero esa palabra sólo significa "conjunto", porque, como se dijo anteriormente, sólo se trata de una codificación de las palabras del vocabulario.

* No hay parecidos entre palabras, ya que son todos vectores ortogonales entre sí.

* Las frases "Esto es un perro, no un gato" y "Esto es un gato, no un perro" tendrían el mismo significado con esta representación.

* La longitud de los vectores suelen ser enormes, ya que el vocabulario también lo es.

## Representaciones por conteo

Estas formas de representar el significado consisten en métodos que, con algún criterio, extraen las características de cada palabra y las representan con un vector de $\mathbb{R}^n$. Por el contrario, las representaciones con modelos neuronales aprenden por sí mismos la forma de realizar la extracción de características, la cual se verá más adelante.

### Matriz de co-ocurrencia

La primera representación distribucional que vamos a ver es de tipo *sparse*, y se basa en contar la cantidad de veces que aparecen las palabras en el contexto de otras, para un determinado corpus de texto.

Supongamos un corpus definido a partir de una serie de documentos *Document 1*, *Document 2*, etc. Con ellos, se define un vocabulario $V$ que contiene a las palabras de todos los documentos. Por otro lado, se define que el contexto de la palabra $w_i$ está determinado por el conjunto de palabras $w_{i-m}, \ldots, w_{i-1}, w_{i+1}, \ldots, w_{i+m}$, es decir, las $m$ palabras anteriores a $w_i$ y las $m$ posteriores. De esta manera, definimos a la matriz $M$ de co-ocurrencia por medio de $[M]_{ij}$ como la cantidad de veces que la palabra $w_j$ apareció en el contexto de $w_i$. Por último, definimos que el vector que representa a la $i$-ésima palabra de $V$ corresponde a la fila o columna $i$ de la matriz de co-ocurrencia.

**Ejemplo 1.** A partir de los siguientes documentos:

Document 1: "all that glitters is not gold"

Document 2: "all is well that ends well"

definimos el vocabulario $V = \{ START, all, that, glitters, is, not, gold, well, ends, END \}$ y la matriz de coocurrencia M para un contexto de tamaño $m=1$:


|     *    | START | all | that | glitters | is   | not  | gold  | well | ends | END |
|----------|-------|-----|------|----------|------|------|-------|------|------|-----|
| START    | 0     | 2   | 0    | 0        | 0    | 0    | 0     | 0    | 0    | 0   |
| all      | 2     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| that     | 0     | 1   | 0    | 1        | 0    | 0    | 0     | 1    | 1    | 0   |
| glitters | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 0    | 0   |
| is       | 0     | 1   | 0    | 1        | 0    | 1    | 0     | 1    | 0    | 0   |
| not      | 0     | 0   | 0    | 0        | 1    | 0    | 1     | 0    | 0    | 0   |
| gold     | 0     | 0   | 0    | 0        | 0    | 1    | 0     | 0    | 0    | 1   |
| well     | 0     | 0   | 1    | 0        | 1    | 0    | 0     | 0    | 1    | 1   |
| ends     | 0     | 0   | 1    | 0        | 0    | 0    | 0     | 1    | 0    | 0   |
| END      | 0     | 0   | 0    | 0        | 0    | 0    | 1     | 1    | 0    | 0   |

Notamos que para las palabras al comienzo y al final de los documentos sólo se consideran parte del contexto las que $m$ de la derecha y las $m$ de la izquierda, respectivamente. 

Por lo tanto, pueden verse algunos ejemplos de vector words definidos a partir de esto:

$$
\begin{align}
w_{all} &= \begin{bmatrix} 2 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \end{bmatrix}^T \\
w_{that} &= \begin{bmatrix} 0 & 1 & 0 & 1 & 0 & 0 & 0 & 1 & 1 & 0 \end{bmatrix}^T \\
\end{align}
$$
$\square$

**Ejemplo 2.** Para un documento se encontró la matriz de coocurrencia, de la cual se muestra una parte:

![alt text](matrix.png)

A partir de ella, pueden graficarse los word embeddings de las palabras *basketball*, *baseball* y *monarchy* en función de las componentes *king* y *player*:

![alt text](plot-coocurrence-matrix.png)

Se observa que, dado que el significado de las palabras *basketball* y *baseball* están más relacionadas con el de *player* que con el de *king*, la componente sobre *player* de ambas es más significativa que la de *king*, y por lo tanto la representación empieza a cobrar sentido. $\square$

### Normalizaciones TF-IDF y PPMI

Uno de los problemas del método anterior es que hay palabras que aparecen frecuentemente en el contexto de otras, pero que "no son importantes". Tal es el caso de los artículos o algunos adjetivos genéricos. Para evitar este tipo de problemas, es posible definir una normalización luego del conteo.

Por ejemplo, si se define a $\mathrm{count}(w_i,w_j)$ como la cantidad de veces que $w_i$ apareció en el contexto de $w_j$ y $d\!f_{i}$ como la cantidad de contextos en que apareció $w_i$, el método TF-IDF define un término

$$
T\!F_{ij} = \mathrm{max} \left\{  \log_{10}\left( \mathrm{count}(w_i,w_j) + 1 \right), 0 \right\}
$$

que no asigna tanta importancia a los palabras con mucha frecuencia, y un término

$$
I\!D\!F_{i} = \log_{10}\left( \frac{N}{d\!f_i} \right)
$$

que ... Con esto, se define que el índice $[M]_{ij}$ de la matriz de co-ocurrencia es ahora

$$
[M]_{ij} = T\!F_{ij} \cdot I\!D\!F_{i}
$$

Otra forma de normalizar el conteo de las plabras es a través de la PPMI (*Positive Puntual Mutual Information*):

$$
PPMI(i,j) = \mathrm{max} \left\{  \log_{10}\left( \frac{\mathrm{count}(w_i,w_j)}{\mathrm{count}(w_i) \mathrm{count}(w_j)} \right), 0 \right\}
$$

### Latent Semantic Analysis

Otra representación del significado puede hacerse simplemente aplicando algún método de reducción de la dimensionalidad a la matriz de co-ocurrencia. 

Paper original: http://lsa.colorado.edu/papers/JASIS.lsi.90.pdf

Material sobre SVD: [1](https://davetang.org/file/Singular_Value_Decomposition_Tutorial.pdf), [2](https://web.stanford.edu/class/cs168/l/l7.pdf), [3](http://theory.stanford.edu/~tim/s15/l/l8.pdf), [4](https://web.stanford.edu/class/cs168/l/l9.pdf), [5](https://en.wikipedia.org/wiki/Singular_value_decomposition#Truncated_SVD) y [6](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html).

## Representaciones con modelos neuronales

En 2003, Bengio[^1] propone un modelo neuronal para representar significado con word embeddings. Después, en 2011, Collobert[^2] porponen un modelo parecido. Estos resultados mostraron la utilidad de este tipo de métodos y marcaron el  inicio de la investigación de este tipo de metodología para resolver tareas de NLP. En la misma línea pero en el 2013, Mikolov[^3] presentó su algoritmo `word2vec` , y luego le sumó un método para entrenarlo eficientemente, llamado "Negative Sample"[^4].

A continuación explicamos el algoritmo `word2vec`, que representa la base para diseñar modelos neuronales y obtener representaciones distribucionales densas.

[^1]: *Bengio, Y., Ducharme, R., Vincent, P., and Jauvin, C. (2003).A neural probabilistic language model.Journal of machinelearning research,3(Feb), 1137–1155*

[^2]: *Collobert, R., Weston,J., Bottou, L., Karlen, M., Kavukcuoglu, K., and Kuksa, P. (2011). Natural language processing (almost) from scratch.JMLR,12, 2493–2537.*

[^3]: *Mikolov,  T.,  Sutskever,  I.,  Chen,  K.,  Corrado,  G.  S.,  andDean, J. (2013a). Distributed representations of words andphrases  and  their  compositionality.   InNIPS  13,   3111–3119.*

[^4]: *Mikolov,   T.,   Chen,   K.,   Corrado,   G.  S.,   and  Dean,   J.(2013). Efficient estimation of word representations in vec-tor space. InICLR 2013.*

### Word2Vec

La intuición básica que formaliza los modelos como `word2vec`es una variante de la hipótesis distribucional: dado que el significado de las palabras queda determinado por el contexto es posible que el significado de la palabra quede aprendido una vez que se conocen todos los contextos en que se puede utilizar esa palabra. Más general aún, si se puede predecir correctamente una palabra dado un contexto o un contexto dado su palabra, entonces se puede probar que el significado de la palabra fue aprendido correctamente. 

A continuación presentamos dos formas de aprender word vectors: *Continuos Bag-Of-Words* (CBOW) y *Skip-gram*. El primero, predice la palabra central a partir de su contexto, mientras que el segundo, las palabras del contexto a partir de la palabra central. En ambos, la premisa básica común es la de asignar una probabilidad a cada suceción de palabras pertenecientes al vocabulario. Por ejemplo, para el vocabulario $V= \{ el, la, gato, salta, saltó, por, sobre, sillón\}$, la oración 



#### Modelos CBOW y Skip-gram básicos

#### Generalización de word2vec

Se define que $V$ es el vocabulario a utilizar, definido a partir de la representación por *one-hot* vectors, y $|V|$ es la cantidad de palabras que contiene éste:

$$
V = \{ h_1, h_2, \ldots, h_{|V|} \}
$$

con 

$$
h_i = 
\begin{bmatrix}
0 \\
\vdots \\
0 \\
1 \\
0 \\
\vdots \\
0
\end{bmatrix}
\in \mathbb{R}^{|V|}
$$

*one-hot* vector en $i$. También se definen las variables aleatorias $\mathbf{w}_1, \ldots, \mathbf{w}_n$  y $\mathbf{c}_1, \ldots, \mathbf{c}_m$, todas ellas con realizaciones en $\mathbb{R}^{|V|}$ que representan la ocurrencia de $n$ palabras centrales y de sus contextos, respectivamente, a través de sus correspondientes vectores *one-hot*. Por ejemplo, para un vocabulario $V=\{ casa, conjunto, el, diez, vacío, la, ellos, ellas, \ldots \}$, las palabras "conjunto" y "vacío" se representan por los vectores $h_2$ y $h_5$, respectivamente. Además, la frase "conjunto vacío" puede armarse de dos maneras posibles: 
1. "conjunto" es la palabra central y "vacío" es su contexto (en cuyo caso $\mathbf{w}$ habrá adoptado el valor $h_2$ y $\mathbf{c}$ el valor $h_5$), o bien
2. "vacío" es la palabra central y "conjunto" es su contexto (en cuyo caso $\mathbf{w}$ habrá adoptado el valor $h_5$ y $\mathbf{c}$ el valor $h_2$).
Para el caso en que se tiene una frase compuesta por más de dos palabras, se suele tomar el promedio de los vectores *one-hot* que conforman la frase para representar el contexto o la palabra central. Por ejemplo, para el caso de la frase "el conjunto vacío" en que se toma a "conjunto" como palabra central y a $\{"\!el\!", "\!vacío\!"\}$ como contexto, se podrá considerar que las variables $\mathbf{w}$ y $\mathbf{c}$ adoptaron los valores $h_2$ y $\frac{1}{2}(h_3 + h_5)$ respectivamente.

Con estas definiciones, establecemos que el objetivo del algoritmo `word2vec` es estimar la probabilidad $P(\mathbf{w}_n=w, \ldots, \mathbf{w}_n=w|\mathbf{c}=c)$ en forma paramétrica y utilizar esos parámetros como representación del significado de las palabras de $V$ (es decir, como word embeddings). Para ello, hay se proponen dos métodos distintos: CBOW y Skip-Gram. 

### Continous Bag of Words (CBOW)

En este caso se define como contexto $c$ a las palabras

### Skip-Gram


## Un término medio: GloVe